In [1]:
import os
os.environ['OPENAI_API_KEY'] = "voc-1420558188126677432688569036a6e03bf58.72907303"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [2]:
%pip install --upgrade --quiet  langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 15.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.4 which is incompatible.


In [3]:


from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [4]:
INSTRUCTION = "Generate a CSV file with at least 10 real estate listing."
SAMPLE_LISTING = \
"""
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
"""

In [5]:
from pydantic import BaseModel, Field, NonNegativeInt
from typing import List
class RealEstateListing(BaseModel):
    Neighborhood: str = Field(description="The neighborhood where the property is located")
    price: NonNegativeInt = Field(description="THe price of the property in USD")
    bedrooms: NonNegativeInt = Field(description="The number of bedrooms in the property")
    bathrooms: NonNegativeInt = Field(description="The number of bathrooms in the property")
    house_size: NonNegativeInt = Field(description="The size of the house in square feet")
    description: str = Field(description="A description of the property")
    neighborhood_description: str = Field(description="A description of the neighborhood")

class ListingCollection(BaseModel):
    listings: List[RealEstateListing] = Field(description="A list of real estate listings")


In [6]:
from langchain_core.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=ListingCollection)

In [7]:
from langchain_core.prompts import PromptTemplate



prompt = PromptTemplate(
    template="{instruction}\n{sample}\n{format_instructions}\n",
    input_variables=["instruction", "sample"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)

print(prompt)

input_variables=['instruction', 'sample'] input_types={} partial_variables={'format_instructions': <bound method PydanticOutputParser.get_format_instructions of PydanticOutputParser(pydantic_object=<class '__main__.ListingCollection'>)>} template='{instruction}\n{sample}\n{format_instructions}\n'


In [8]:
query = prompt.format(instruction=INSTRUCTION, sample=SAMPLE_LISTING)
print(query)

Generate a CSV file with at least 10 real estate listing.

Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bik

In [9]:
response = llm.invoke(query)

In [10]:
print(response.content)

{
  "listings": [
    {
      "Neighborhood": "Green Oaks",
      "price": 800000,
      "bedrooms": 3,
      "bathrooms": 2,
      "house_size": 2000,
      "description": "Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.",
      "neighborhood_description": "Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public 

In [11]:
import pandas as pd

result = parser.parse(response.content)
df = pd.DataFrame([listing.model_dump() for listing in result.listings])
df.head()


# METHOD 2
# import json
# data = json.loads(result.json())
# df = pd.DataFrame(data["listings"])
# df.head()

,Neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description
0,Green Oaks,800000,3,2,2000,Welcome to this eco-friendly oasis nestled in ...,"Green Oaks is a close-knit, environmentally-co..."
1,Sunnyvale,950000,4,3,2500,Located in the desirable neighborhood of Sunny...,"Sunnyvale is known for its top-rated schools, ..."
2,Brooklyn Heights,1200000,5,4,3500,"Welcome to this stunning 5-bedroom, 4-bathroom...",Brooklyn Heights is known for its tree-lined s...
3,Pacific Palisades,3500000,6,5,5000,Located in the prestigious neighborhood of Pac...,Pacific Palisades is known for its upscale hom...
4,Back Bay,2500000,4,3,3000,"Welcome to this elegant 4-bedroom, 3-bathroom ...","Back Bay is known for its upscale homes, tree-..."


In [12]:
df.to_csv('real_sestate_listings.csv', index_label='id')

In [13]:
import os
os.environ['OPENAI_API_KEY'] = "voc-1420558188126677432688569036a6e03bf58.72907303"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [14]:
%pip install --upgrade --quiet  langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.4 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.


In [15]:
%pip install --upgrade --quiet chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.3 MB/s eta 

In [16]:
import shutil
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import pandas as pd
import os

CHROMA_PATH = "/content/chroma"
CSV_PATH = "real_sestate_listings.csv"

embedding_function = OpenAIEmbeddings()

df = pd.read_csv(CSV_PATH)
documents = []
for index, row in df.iterrows():
    documents.append(Document(page_content=row['description'], metadata={'id': str(index)}))

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)

chunks = text_splitter.split_documents(documents)
print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

if chunks:
    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

db = Chroma.from_documents(chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH)


db.persist()
print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}")

Split 10 documents into 22 chunks.
gourmet kitchen boasts custom cabinetry, high-end appliances, and a breakfast nook. Retreat to the luxurious master suite with a spa-like bathroom and walk-in closet. The landscaped backyard offers a tranquil retreat with a fountain and outdoor dining area.
{'id': '4', 'start_index': 195}
Saved 22 chunks to /content/chroma


/tmp/ipython-input-308074583.py:40: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [18]:
# ✅ Create and persist the Chroma vector database

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# Initialize embedding model
embeddings = OpenAIEmbeddings()

# Define directory for persistence (you already use CHROMA_PATH)
persist_directory = CHROMA_PATH

# Assuming your documents are stored in a variable called `docs`
db = Chroma.from_documents(chunks, embeddings, persist_directory=persist_directory)

# Persist the database
db.persist()
print("✅ Vector database created and saved at:", persist_directory)

✅ Vector database created and saved at: /content/chroma


In [19]:
query_text = "A comfortable three-bedroom house with a spacious kitchen and a cozy living room."

BASIC_PROMPT_TEMPLATE = \
"""
Based on the following context:

{context}


-----

Answer the question : {question}
"""

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


def predict_response(query_text, PROMPT_TEMPLATE):
    embedding_function = OpenAIEmbeddings()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_relevance_scores(query_text, k=3)
    if len(results) == 0 or results[0][1] < 0.7:
        print(f"Unable to find matching results.")
    else:
        context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
        prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
        prompt = prompt_template.format(context=context_text, question=query_text)
        print(f"Generated Prompt:\n{prompt}")

        model = ChatOpenAI()
        response_text = model.invoke(prompt).content
        sources = [doc.metadata.get("id", None) for doc, _score in results]
        formatted_response = f"Response: {response_text}\nSources: {sources}"
        print(formatted_response)

In [21]:
predict_response(query_text, BASIC_PROMPT_TEMPLATE)

/tmp/ipython-input-2526889268.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)


Generated Prompt:
Human: 
Based on the following context:

for hosting dinner parties. The gourmet kitchen features high-end appliances and a breakfast nook overlooking the landscaped backyard. Retreat to the luxurious master suite with a private balcony and spa-like bathroom. This elegant home combines classic charm with modern amenities.

---

for hosting dinner parties. The gourmet kitchen features high-end appliances and a breakfast nook overlooking the landscaped backyard. Retreat to the luxurious master suite with a private balcony and spa-like bathroom. This elegant home combines classic charm with modern amenities.

---

a gourmet kitchen, and a spacious dining area. The master suite includes a spa-like bathroom and walk-in closet. Relax on the private balcony with stunning views of the city skyline and waterfront.


-----

Answer the question : A comfortable three-bedroom house with a spacious kitchen and a cozy living room.

Response: Based on the context given, the comfortab

In [22]:
AUGMENT_PROMPT_TEMPLATE =\
"""
Based on the following context:

{context}

---

craft a response that not only answers the question {question}, but also ensures that your explanation is distinct, captivating, and customized to align with the specified preferences. This involves subtly emphasizing aspects of the property that align with what the buyer is looking for.
"""

In [23]:
predict_response(query_text, AUGMENT_PROMPT_TEMPLATE)

Generated Prompt:
Human: 
Based on the following context:

for hosting dinner parties. The gourmet kitchen features high-end appliances and a breakfast nook overlooking the landscaped backyard. Retreat to the luxurious master suite with a private balcony and spa-like bathroom. This elegant home combines classic charm with modern amenities.

---

for hosting dinner parties. The gourmet kitchen features high-end appliances and a breakfast nook overlooking the landscaped backyard. Retreat to the luxurious master suite with a private balcony and spa-like bathroom. This elegant home combines classic charm with modern amenities.

---

a gourmet kitchen, and a spacious dining area. The master suite includes a spa-like bathroom and walk-in closet. Relax on the private balcony with stunning views of the city skyline and waterfront.

---

craft a response that not only answers the question A comfortable three-bedroom house with a spacious kitchen and a cozy living room., but also ensures that yo

In [ ]:
pip freeze > requirements.txt
